# Worthäufigkeiten und Wordclouds

Die linguistisch analysierten Daten hast du nun in der Datenbank abgespeichert. Jetzt kannst du mit der Analyse starten.

## Daten selektieren

Aus der Datenbank liest du im ersten Schritt die analysierten Daten aus:

In [ ]:
!pip install textacy

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles ORDER BY datePublished", sql, index_col="id")

Um die Wörter zu rekonstruieren, definierst du eine Funktion, die die Tokens aus den Spalte mittels `regex` rekonstruieren kann.

In [ ]:
import regex as re

def single_words(df, field):
    return [w for words in df[field] for w in re.split(r'\||\#', words)]

Für die Analyse benötigst du den `Counter` aus dem `collections`-Paket. Die einzelnen Wortarten oder deren Kombinationen hast du im letzten Schritt in der Datenbank gespeichert:

In [ ]:
from collections import Counter

nouns = Counter(single_words(df, "nouns"))
adjectives = Counter(single_words(df, "adjectives"))
verbs = Counter(single_words(df, "verbs"))
nav = Counter(single_words(df, "nav"))
lemmas = Counter(single_words(df, "lemmas"))

Auch die *Noun Chunks* und die *Entitäten* kannst du so rekonstruieren:

In [ ]:
noun_chunks = Counter(single_words(df, "noun_chunks"))
entities = Counter(single_words(df, "entities"))

Lass dir nun die häufigsten Substantive anzeigen:

In [ ]:
nouns.most_common()

Eine deutlich attraktivere Darstellung erreichst du mit dem `wordcloud`-Paket:

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nouns)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

Offenbar sind im Newsticker viele Unternehmens-Meldungen enthalten!

Die Verben sind weniger aufschlussreich, weil sie zu *generisch* sind:

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(verbs)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

Auch die Adjektive sind nur bedingt aussagekräftig:

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(adjectives)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

Die Kombinationen der Wörter sind leider stark von den Verben dominiert:

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nav)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

Um etwas genauere Informationen zu erhalten, wendest du einen Trick an und setzt die Häufigkeiten der Stopwords auf 0:

In [ ]:
from spacy.lang.de.stop_words import STOP_WORDS
for w in STOP_WORDS:
    nav[w] = 0

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(nav)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

Genauso kannst du noch die *Noun Chunks* und die *Entitäten* auswerten:

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(noun_chunks)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

In [ ]:
wc = WordCloud(background_color="white", max_words=100)
wc.generate_from_frequencies(entities)
plt.figure(figsize=(12,12))
plt.imshow(wc, interpolation='bilinear')
plt.axis("off");

## Viele Auswertungen

Das waren jetzt ganz schön viele Auswertungen! Nicht immer musst und solltest du alle durchführen. Insgesamt verschaffen dir Wordclouds einen gute qualitativen Eindruck, welche Informationen in einem Korpus enthalten sind